# Time-Resolved ODMR Lab

## Setup

In [ ]:
# Libraries
import pyvisa                                        # Communication (also install pyvisa-py or NI VISA)
import matplotlib.pyplot as plt  # Plots
import numpy as np                              # Maths
import pandas as pd                            # DataFrames
import time                                            # Delays
import tqdm                                            # Progress bars (use 'for i in tqdm.tqdm(iter)')
import os                                                 # Used for determining platform

In [ ]:
# Device drivers
from Devices.LockIn import SR830M
from Devices.PicoPulse import PicoPulse
from Devices.LO import KuhnePLL

### Definitions

In [ ]:
# Windows
# Find the COM port addresses of all three devices involved.
if os.name == 'nt':
    lockin_com = 8
    pico_com = 3
    osc_com = 4
    
    lockin_addr = f'ASRL{lockin_com}::INSTR'
    pico_addr = f'ASRL{pico_com}::INSTR'
    osc_addr = f'COM{osc_com}'

# Linux and MacOS
# Find the device files of the involved devices.
if os.name == 'posix':
    lockin_dev = '/dev/ttyUSB0'
    pico_dev = '/dev/ttyACM0'
    osc_dev = '/dev/ttyACM1'
        
    lockin_addr = f'ASRL{lockin_dev}::INSTR'
    pico_addr = f'ASRL{pico_dev}::INSTR'
    osc_addr = osc_dev

In [ ]:
# This may also help
rm = pyvisa.ResourceManager()
print(rm.list_resources())
rm.close()

In [ ]:
# pico-pulse pin mapping
# This is device-specific, you probably don't need to touch this.
pico_pins = {
    'lockin': 'ch1',
    'Q': 'ch2',
    'I': 'ch3',
    'laser': 'ch4'
}

## Basic usage

### pico-pulse

In [ ]:
# Turn off laser

# Always instantiate a new resource manager and device object when starting a new cell
rm = pyvisa.ResourceManager()
pico = PicoPulse(rm, pico_addr, pico_pins)

idle_seq = pd.DataFrame(
        columns = ['time', 'lockin', 'laser'],
        data = [
            [1e6, 0, 0],
            [1e6, 1, 0],
        ]
    )

pico.sendSequence(idle_seq)

# Don't forget to close the resource manager
rm.close()

In [ ]:
# Turn on laser

# Always instantiate a new resource manager and device object when starting a new cell
rm = pyvisa.ResourceManager()
pico = PicoPulse(rm, pico_addr, pico_pins)

adjust_seq = pd.DataFrame(
        columns = ['time', 'lockin', 'laser'],
        data = [
            [1e6, 0, 1],
            [1e6, 1, 1],
        ]
    )

pico.sendSequence(adjust_seq)

# Don't forget to close the resource manager
rm.close()

### SR830(M) lock-in amplifier

In [ ]:
# Read single values from the lock-in

# Always instantiate a new resource manager and device object when starting a new cell
rm = pyvisa.ResourceManager()
lockin = SR830M(rm, lockin_addr)

# You can read just a single value
ref = lockin.snapshot('ref' )

# Or a list of values, up to 6 elements long
x, y, r, theta = lockin.snapshot(['x', 'y', 'r', 'theta'])

# Don't forget to close the resource manager
rm.close()
x, y, r, theta, ref

In [ ]:
# Read out multiple values automatically.
# Useful for taking multiple measurements to integrate and calculate uncertainty.

# Always instantiate a new resource manager and device object when starting a new cell
rm = pyvisa.ResourceManager()
lockin = SR830M(rm, lockin_addr)

# Sample X and Y for 8 seconds with an automatic sample rate
# (calculated from time constant)
xs, ys = lockin.multiRead('x', 'y', 8)

# Sample AUX4 for 10 seconds at 4 Hz
# Setting channel 1 to None speeds up readout
_, aux4s = lockin.multiRead(None, 'aux4', 10, 4)

# Don't forget to close the resource manager
rm.close()
xs, ys, aux4s

### MW oscillator

In [ ]:
# Instantiate a new one in each cell
osc = KuhnePLL(osc_addr)

# Self-explanatory. setMHz, set kHz and setHz methods are also available.
osc.setGHz(2.87)

# Make sure you delete it properly when you're done
del osc